# TSS_bed_generate.ipynb
## This script takes in refseq annotation files of genes <br> and genes filtered for start codons and calculates <br> a window around either the 5' end of the gene <br> annotation or the start codon
## This is the script used for making GTFs for counting hg38 and mm10 <br> genes in DBNascent, but has been updated for future GTFs <br> (See transcript_cluster_ends_v1.0.py) - the only differences are that this <br> script does not include transcripts exactly 100 bases apart <br> and has some differences in which transcript ID is used for those where <br> multiple transcripts match coordinates

### Import libraries

In [125]:
import math

import numpy as np
import pandas as pd

### Define refseq input, output basename, and desired TSS window

In [126]:
file1 = ["/home/zarko/Data/annotation/" "hg38_refseq_genenames_included.bed"]
basename = "hg38_refseq"
window = 50

### Dump input files into variables

In [127]:
with open(file1[0]) as f:
    regions = []
    for line in f:
        reg = tuple(line.strip().split("\t"))
        regions.append(reg)

### Depending on accession (NM or NR), take most 5' coordinate <br> based on strand and (for NM) start codon, calculate windows <br> around them, and store windows in a printable bed format

In [128]:
ts_ends = []

for i in range(len(regions)):

    # Don't use alternative chromosome builds
    if "_" in regions[i][0]:
        continue

    # Find strand
    strand = regions[i][5]

    # Find most 5' coordinate of whole gene annotation or start codon
    if strand == "+":
        five_prime = int(regions[i][1])
        three_prime = int(regions[i][2])
    else:
        five_prime = int(regions[i][2])
        three_prime = int(regions[i][1])

    # Calculate windows around coordinates and store in new list
    #    low5 = int(five_prime - window/2)
    #    high5 = int(five_prime + window/2)
    #    low3 = int(three_prime - window/2)
    #    high3 = int(three_prime + window/2)

    #    nr = [regions[i][0],low5,high5,low3,high3,regions[i][5],regions[i][12]]
    nr = [
        regions[i][0],
        five_prime,
        three_prime,
        regions[i][5],
        regions[i][3],
        regions[i][12],
    ]
    ts_ends.append(nr)

transcript_ends = pd.DataFrame(ts_ends)
transcript_ends = transcript_ends.rename(
    columns={
        0: "chrom",
        1: "fivep",
        2: "threep",
        3: "strand",
        4: "transcriptID",
        5: "geneID",
    }
)
# transcript_ends = transcript_ends.rename(columns={0:"chrom",1:"low5",2:"high5",3:"low3",
#                                4:"high3",5:"strand",6:"transcriptID",7:"geneID"})

### Collapse overlaps in the 50 base windows

In [129]:
uniq_ts_ends = transcript_ends.drop_duplicates().sort_values(
    by=["geneID", "chrom", "fivep"]
)
uniq_genes = uniq_ts_ends["geneID"].drop_duplicates()
collapsed_ts_ends = []

for i in range(len(uniq_genes)):
    ts_working = uniq_ts_ends[uniq_ts_ends["geneID"] == uniq_genes.iloc[i]]
    bounds = [[ts_working["fivep"].iloc[0], ts_working["threep"].iloc[0]]]
    if ts_working["strand"].iloc[0] == "+":
        collapsed_ts_ends.append(
            [
                ts_working["chrom"].iloc[0],
                "hg38_refseq",
                "gene_length",
                str(ts_working["fivep"].iloc[0]),
                str(ts_working["threep"].iloc[0]),
                "0",
                ts_working["strand"].iloc[0],
                ".",
                str(
                    'gene_id "{}"; transcript_id "{}"'.format(
                        ts_working["geneID"].iloc[0], ts_working["transcriptID"].iloc[0]
                    )
                ),
            ]
        )
    else:
        collapsed_ts_ends.append(
            [
                ts_working["chrom"].iloc[0],
                "hg38_refseq",
                "gene_length",
                str(ts_working["threep"].iloc[0]),
                str(ts_working["fivep"].iloc[0]),
                "0",
                ts_working["strand"].iloc[0],
                ".",
                str(
                    'gene_id "{}"; transcript_id "{}"'.format(
                        ts_working["geneID"].iloc[0], ts_working["transcriptID"].iloc[0]
                    )
                ),
            ]
        )

    if len(ts_working) > 1:
        for j in range(1, len(ts_working)):
            curr_bounds = [ts_working["fivep"].iloc[j], ts_working["threep"].iloc[j]]
            add_to_bounds = 0
            for k in range(len(bounds)):
                if (curr_bounds[0] < (bounds[k][0] - 100)) | (
                    curr_bounds[0] > (bounds[k][0] + 100)
                ):
                    add_to_bounds = 1
                elif (curr_bounds[1] < (bounds[k][1] - 100)) | (
                    curr_bounds[1] > (bounds[k][1] + 100)
                ):
                    add_to_bounds = 1
                else:
                    add_to_bounds = 0
                    break
            if add_to_bounds == 1:
                bounds.append(curr_bounds)
                if ts_working["strand"].iloc[j] == "+":
                    collapsed_ts_ends.append(
                        [
                            ts_working["chrom"].iloc[j],
                            "hg38_refseq",
                            "gene_length",
                            str(curr_bounds[0]),
                            str(curr_bounds[1]),
                            "0",
                            ts_working["strand"].iloc[j],
                            ".",
                            str(
                                'gene_id "{}"; transcript_id "{}"'.format(
                                    ts_working["geneID"].iloc[j],
                                    ts_working["transcriptID"].iloc[j],
                                )
                            ),
                        ]
                    )
                else:
                    collapsed_ts_ends.append(
                        [
                            ts_working["chrom"].iloc[j],
                            "hg38_refseq",
                            "gene_length",
                            str(curr_bounds[1]),
                            str(curr_bounds[0]),
                            "0",
                            ts_working["strand"].iloc[j],
                            ".",
                            str(
                                'gene_id "{}"; transcript_id "{}"'.format(
                                    ts_working["geneID"].iloc[j],
                                    ts_working["transcriptID"].iloc[j],
                                )
                            ),
                        ]
                    )

#    five_prime_intervals = pd.arrays.IntervalArray.from_arrays(list(ts_working['low5']),list(ts_working['high5']))
#    three_prime_intervals = pd.arrays.IntervalArray.from_arrays(ts_working['low3'],ts_working['high3'])

### Make outfile names

In [130]:
indir = file1[0].split("/")
outdir = "/".join(indir[0:-1])
outfile = "".join([outdir, "/", basename, "_diff53prime.gtf"])

### Export data

In [131]:
with open(outfile, "wt") as f:
    for i in range(len(collapsed_ts_ends)):
        entry = "\t".join(collapsed_ts_ends[i])
        f.write("".join([entry, "\n"]))